In [1]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [2]:
hg_model_hub_name = "alisawuffles/roberta-large-wanli"
nli_model = NLIModel(hg_model_hub_name)

In [6]:
# MAIN GOAL


boil_caesium_plan = """
    IF your goal is to boil caesium THEN
        move to kitchen,
        pick up thermometer,
        get metal pot,
        get caesium,
        heat caesium on blast furnace
"""

# get metal pot
subplan_a = """
    IF your goal is to get metal pot CONSIDERING you are in the kitchen AND you see a cupboard THEN
        open cupboard,
        pick up metal pot
"""

# fill metal pot with water
subplan_b = """
    IF your goal is to get caesium CONSIDERING you have metal pot in your inventory THEN
        move to workshop,
        pick up caesium,
        focus on caesium,
        move caesium to metal pot
"""

# heat water on stove
subplan_c = """
    IF your goal is to heat caesium on blast furnace CONSIDERING you have metal pot with caesium in your inventory THEN
        move to foundry,
        open blast furnace,
        move metal pot to blast furnace,
        activate blast furnace,
        use thermometer on metal pot,
        wait,
        use thermometer on metal pot
"""

all_plans = [boil_caesium_plan, subplan_a, subplan_b, subplan_c]

pl = PlanLibrary()
pl.load_plans_from_strings(all_plans)  # load plans from strings above
pl.load_plans_from_file("plans.txt")  # load plans from file
print(pl.plans.keys())

dict_keys(['boil caesium', 'get metal pot', 'get caesium', 'heat caesium on blast furnace', 'move to art studio', 'move to bedroom', 'move to greenhouse', 'move to kitchen', 'move to living room', 'move to workshop', 'move to outside', 'move to foundry', 'move to bathroom', 'move to hallway'])


In [7]:
env = ScienceWorldEnv("", "", envStepLimit=100)

#root_event = 'use chemistry to create green paint'

task = 'boil'
env.load(task, 0)
#randVariationIdx = env.getRandomVariationTest()
randVariationIdx = 29
env.load(task, randVariationIdx)

goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])

goal

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Task Name: boil variation 29
Task Description: Your task is to boil caesium. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


'boil caesium'

In [8]:
def step_function(action: str) -> State:
    observation, reward, isCompleted, info = env.step(action)
    updated_state = parse_observation(observation=observation,
                                      inventory=info['inv'],
                                      look_around=info['look'],
                                      task=goal,
                                      valid_actions=info['valid'],
                                      score=info['score'])
    print(f"Action: {action} -> Obs {observation}")
    return updated_state


agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.reward)

event -> boil caesium -> state False
event -> move to kitchen -> state False
Action: open door to kitchen -> Obs The door is now open.
Action: go to kitchen -> Obs You move to the kitchen.
Action: pick up thermometer -> Obs You move the thermometer to the inventory.
event -> get metal pot -> state False
Action: open cupboard -> Obs The cupboard is now open.
Action: pick up metal pot -> Obs You move the metal pot to the inventory.
event -> get caesium -> state False
event -> move to workshop -> state False
Action: open door to hallway -> Obs The door is now open.
Action: go to hallway -> Obs You move to the hallway.
Action: open door to workshop -> Obs The door is now open.
Action: go to workshop -> Obs You move to the workshop.
Action: pick up caesium -> Obs You move the caesium to the inventory.
Action: focus on caesium -> Obs You focus on the caesium.
Action: move caesium to metal pot -> Obs You move the caesium to the metal pot.
event -> heat caesium on blast furnace -> state False
